# 1. 환경 세팅

In [1]:
import boto3
region = boto3.Session().region_name
opensearch = boto3.client('opensearch', region)

%store -r opensearch_user_id opensearch_user_password domain_name opensearch_domain_endpoint

try:
    opensearch_user_id
    opensearch_user_password
    domain_name
    opensearch_domain_endpoint
   
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] Run 00_setup notebook first or Create Your Own OpenSearch Domain")
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [2]:
%load_ext autoreload
%autoreload 2

import sys, os
module_path = ".."
sys.path.append(os.path.abspath(module_path))

## Bedrock Client 생성
### 선수 지식
아래의 노트북을 먼저 실행해서, Bedrock 에 접근 가능하게 합니다.
- amazon-bedrock-workshop-webinar-kr/00_Setup/setup.ipynb

In [3]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

# print (colored("\n== FM lists ==", "green"))
# pprint (bedrock_info.get_list_fm_models())

bedrock = boto3.client(service_name='bedrock')
model_list = bedrock.list_foundation_models()
result = [(fm_list["modelName"], fm_list["modelId"]) for fm_list in model_list["modelSummaries"] if fm_list['inferenceTypesSupported'] == ['ON_DEMAND']]
pprint(result)

Create new client
  Using region: None
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-west-2.amazonaws.com)
[('Titan Text Large', 'amazon.titan-tg1-large'),
 ('Titan Text Embeddings v2', 'amazon.titan-embed-g1-text-02'),
 ('Titan Text G1 - Lite', 'amazon.titan-text-lite-v1'),
 ('Titan Text G1 - Express', 'amazon.titan-text-express-v1'),
 ('Titan Embeddings G1 - Text', 'amazon.titan-embed-text-v1'),
 ('Titan Multimodal Embeddings G1', 'amazon.titan-embed-image-v1'),
 ('Titan Image Generator G1', 'amazon.titan-image-generator-v1'),
 ('SDXL 0.8', 'stability.stable-diffusion-xl'),
 ('SDXL 0.8', 'stability.stable-diffusion-xl-v0'),
 ('SDXL 1.0', 'stability.stable-diffusion-xl-v1'),
 ('J2 Grande Instruct', 'ai21.j2-grande-instruct'),
 ('J2 Jumbo Instruct', 'ai21.j2-jumbo-instruct'),
 ('Jurassic-2 Mid', 'ai21.j2-mid'),
 ('Jurassic-2 Mid', 'ai21.j2-mid-v1'),
 ('Jurassic-2 Ultra', 'ai21.j2-ultra'),
 ('Jurassic-2 Ultra', 'ai21.j2-ultra

---

## `InvokeModel` body and output

The `invoke_model()` method of the Amazon Bedrock runtime client (`InvokeModel` API) will be the primary method we use for most of our Text Generation and Processing tasks - whichever model we're using.

Although the method is shared, the format of input and output varies depending on the foundation model used - as described below:

### Anthropic Claude

#### Input

```json
{
    "prompt": "\n\nHuman:<prompt>\n\nAnswer:",
    "max_tokens_to_sample": 300,
    "temperature": 0.5,
    "top_k": 250,
    "top_p": 1,
    "stop_sequences": ["\n\nHuman:"]
}
```

#### Output

```json
{
    "completion": "<output>",
    "stop_reason": "stop_sequence"
}
```

### Amazon Titan Large

#### Input
```json
{   
    "inputText": "<prompt>",
    "textGenerationConfig" : { 
        "maxTokenCount": 512,
        "stopSequences": [],
        "temperature": 0.1,  
        "topP": 0.9
    }
}
```

#### Output

```json
{
    "inputTextTokenCount": 613,
    "results": [{
        "tokenCount": 219,
        "outputText": "<output>"
    }]
}
```

### AI21 Jurassic (Grande and Jumbo) 

#### Input

```json
{
    "prompt": "<prompt>",
    "maxTokens": 200,
    "temperature": 0.5,
    "topP": 0.5,
    "stopSequences": [],
    "countPenalty": {"scale": 0},
    "presencePenalty": {"scale": 0},
    "frequencyPenalty": {"scale": 0}
}
```

#### Output

```json
{
    "id": 1234,
    "prompt": {
        "text": "<prompt>",
        "tokens": [
            {
                "generatedToken": {
                    "token": "\u2581who\u2581is",
                    "logprob": -12.980147361755371,
                    "raw_logprob": -12.980147361755371
                },
                "topTokens": null,
                "textRange": {"start": 0, "end": 6}
            },
            //...
        ]
    },
    "completions": [
        {
            "data": {
                "text": "<output>",
                "tokens": [
                    {
                        "generatedToken": {
                            "token": "<|newline|>",
                            "logprob": 0.0,
                            "raw_logprob": -0.01293118204921484
                        },
                        "topTokens": null,
                        "textRange": {"start": 0, "end": 1}
                    },
                    //...
                ]
            },
            "finishReason": {"reason": "endoftext"}
        }
    ]
}
```

### Stability AI Stable Diffusion XL

#### Input

```json
{
    "text_prompts": [
        {"text": "this is where you place your input text"}
    ],
    "cfg_scale": 10,
    "seed": 0,
    "steps": 50
}
```

#### Output

```json
{ 
    "result": "success", 
    "artifacts": [
        {
            "seed": 123, 
            "base64": "<image in base64>",
            "finishReason": "SUCCESS"
        },
        //...
    ]
}
```

# 2. Titan Embedding 및 LLM 인 Claude-v2 모델 로딩

## LLM 로딩 (Claude-v2-1)

In [4]:
from botocore.config import Config

session = boto3.Session()

retry_config = Config(
    region_name=os.environ.get("AWS_DEFAULT_REGION", None),
    retries={
        "max_attempts": 10,
        "mode": "standard",
    },
)


bedrock_runtime = session.client(
    service_name="bedrock-runtime",
    config=retry_config
)

In [5]:
# If you'd like to try your own prompt, edit this parameter!
prompt_data = """Human: 나는 인공지능 AI 보험 서비스입니다. 생명과 손해 보험의 차이에 대해 설명해 주세요.

Assistant:
"""


In [6]:
from IPython.display import clear_output, display, display_markdown, Markdown

body = json.dumps({"prompt": prompt_data, "max_tokens_to_sample": 500})
modelId = "anthropic.claude-instant-v1"  # (Change this to try different model versions)
accept = "application/json"
contentType = "application/json"

try:

    response = bedrock_runtime.invoke_model_with_response_stream(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    stream = response.get('body')
    output = []

    if stream:
        for event in stream:
            chunk = event.get('chunk')
            if chunk:
                chunk_obj = json.loads(chunk.get('bytes').decode())
                text = chunk_obj['completion']
                clear_output(wait=True)
                output.append(text)
                display_markdown(Markdown(''.join(output)))

except botocore.exceptions.ClientError as error:

    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")

    else:
        raise error


 생명보험과 손해보험은 주로 아래와 같이 차이가 있습니다:

- 생명보험의 경우 사람의 생명이나 건강에 대한 위험을 보장합니다. 즉, 피보험자가 사망하는 경우나 특정 질환이 생기는 경우 등에 지급금을 줍니다. 

- 손해보험은 사람이나 재산이 수배되는 손해를 보장합니다. 예를 들어 자동차보험의 경우 교통사고로 인한 자동차 수선비나 책임보험 등을 보장합니다.

- 생명보험은 피보험자의 사망시나 요양비 등 지급금을 줍니다. 손해보험은 실제 발생한 손해액을 보상합니다.

- 생명보험은 보험기간 동안 보험료를 지속적으로 납입해야 하지만, 손해보험은 사고 발생시 보상금을 지급합니다.

이러한 점에서 생명보험은 개인의 삶과 관련한 위험을, 손해보험은 재산과 관련한 위험을 각각 보장하는 것이 주요 차이점입니다.

In [7]:
from langchain.llms.bedrock import Bedrock
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [8]:
# - create the Anthropic Model
llm_text = Bedrock(
    model_id=modelId,
    client=boto3_bedrock,
    model_kwargs={
        "max_tokens_to_sample": 512
    },
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)
llm_text

Bedrock(client=<botocore.client.BedrockRuntime object at 0x7f12101984f0>, model_id='anthropic.claude-instant-v1', model_kwargs={'max_tokens_to_sample': 512}, streaming=True, callbacks=[<langchain_core.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x7f11eb53ac20>])

In [9]:
# If you'd like to try your own prompt, edit this parameter!
prompt_template = """

Human: 나는 인공지능 AI 보험 서비스입니다. 생명과 손해 보험의 차이에 대해 설명해 주세요.

<context>
{history}
</context>

Question: {input} using within <context></context>


Assistant:
"""

In [10]:
from langchain.prompts import PromptTemplate

PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=['history', 'input']
)

In [11]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

conversation = ConversationChain(
    llm=llm_text, 
    verbose=True, 
    memory=ConversationBufferMemory(),
    prompt=PROMPT
)

conversation.predict(input=prompt_data)



> Entering new ConversationChain chain...
Prompt after formatting:


Human: 나는 인공지능 AI 보험 서비스입니다. 생명과 손해 보험의 차이에 대해 설명해 주세요.

<context>

</context>

Question: Human: 나는 인공지능 AI 보험 서비스입니다. 생명과 손해 보험의 차이에 대해 설명해 주세요.

Assistant:
 using within <context></context>


Assistant:



/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/langchain/llms/bedrock.py:52: UserWarning: Error: Prompt must alternate between '

Human:' and '

Assistant:'. Received 

Human: 나는 인공지능 AI 보험 서비스입니다. 생명과 손해 보험의 차이에 대해 설명해 주세요.

<context>

</context>

Question: 

Human: 나는 인공지능 AI 보험 서비스입니다. 생명과 손해 보험의 차이에 대해 설명해 주세요.

Assistant:
 using within <context></context>


Assistant:

  warnings.warn(ALTERNATION_ERROR + f" Received {input_text}")
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/langchain/llms/bedrock.py:57: UserWarning: Error: Prompt must alternate between '

Human:' and '

Assistant:'. Received 

Human: 나는 인공지능 AI 보험 서비스입니다. 생명과 손해 보험의 차이에 대해 설명해 주세요.

<context>

</context>

Question: 

Human: 나는 인공지능 AI 보험 서비스입니다. 생명과 손해 보험의 차이에 대해 설명해 주세요.

Assistant:
 using within <context></context>


Assistant:

  warnings.warn(ALTERNATION_ERROR + f" Received {input_text}")


 생명 보험과 손해 보험의 주요한 차이점은 아래와 같습니다:

- 생명 보험은 보험 가입자의 사망시 사망보장금을 지급합니다. 즉, 생명 보험은 개인의 생명을 대상으로 합니다. 

- 손해 보험은 사고로 인한 재산 손실이나 배상책임을 보상합니다. 예를 들어 자동차 손해 보험은 교통사고로 발생한 자동차 수리비용을, 화재 손해 보험은 화재로 인한 건조물의 손실을 각각 보상합니다.

- 생명 보험은 보험 기간 동안 보험 가입자가 살아있을 동안 보험 사고가 발생하지 않는 이상 보상금을 지급하지 않습니다. 손해 보험은 보험 사고가 발생했을 경우 보상금을 지급합니다.

- 생명 보험은 개인 생명을, 손해 보험은 재산을 대상으로 하므로 가입 목적이 다릅니다.

이상이 생명 보험과 손해 보험의 주요 차이점입니다. 도움이 되었기를 바랍니다.
> Finished chain.


' 생명 보험과 손해 보험의 주요한 차이점은 아래와 같습니다:\n\n- 생명 보험은 보험 가입자의 사망시 사망보장금을 지급합니다. 즉, 생명 보험은 개인의 생명을 대상으로 합니다. \n\n- 손해 보험은 사고로 인한 재산 손실이나 배상책임을 보상합니다. 예를 들어 자동차 손해 보험은 교통사고로 발생한 자동차 수리비용을, 화재 손해 보험은 화재로 인한 건조물의 손실을 각각 보상합니다.\n\n- 생명 보험은 보험 기간 동안 보험 가입자가 살아있을 동안 보험 사고가 발생하지 않는 이상 보상금을 지급하지 않습니다. 손해 보험은 보험 사고가 발생했을 경우 보상금을 지급합니다.\n\n- 생명 보험은 개인 생명을, 손해 보험은 재산을 대상으로 하므로 가입 목적이 다릅니다.\n\n이상이 생명 보험과 손해 보험의 주요 차이점입니다. 도움이 되었기를 바랍니다.'

## Function call

In [12]:
from utils.function_call import add_tools, run_loop

In [27]:
def create_prompt(tools_string, user_input):
    prompt_template = f"""
In this environment you have access to a set of tools you can use to answer the user's question.

You may call them like this. Only invoke one function at a time and wait for the results before invoking another function:
<function_calls>
<invoke>
<tool_name>$TOOL_NAME</tool_name>
<parameters>
<$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>
...
</parameters>
</invoke>
</function_calls>

Here are the tools available:
<tools>
{tools_string}
</tools>

Human:
{user_input}


Assistant:
"""
    return prompt_template

In [29]:
user_input = "Can you check the weather for me in Paris, France?"
tools_string = add_tools()
prompt = create_prompt(tools_string, user_input)
run_loop(prompt)


In this environment you have access to a set of tools you can use to answer the user's question.

You may call them like this. Only invoke one function at a time and wait for the results before invoking another function:
<function_calls>
<invoke>
<tool_name>$TOOL_NAME</tool_name>
<parameters>
<$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>
...
</parameters>
</invoke>
</function_calls>

Here are the tools available:
<tools>

            <tool_description>
            <tool_name>get_weather</tool_name>
            <description>
            Returns weather data for a given latitude and longitude. </description>
            <parameters>
            <parameter>
            <name>latitude</name>
            <type>string</type>
            <description>The latitude coordinate as a string</description>
            </parameter> <parameter>
            <name>longitude</name>
            <type>string</type>
            <description>The longitude coordinate as a string</description>
         

In [30]:
user_input = "한국의 오늘 날씨는 어떻게 되나요?"
tools_string = add_tools()
prompt = create_prompt(tools_string, user_input)
run_loop(prompt)


In this environment you have access to a set of tools you can use to answer the user's question.

You may call them like this. Only invoke one function at a time and wait for the results before invoking another function:
<function_calls>
<invoke>
<tool_name>$TOOL_NAME</tool_name>
<parameters>
<$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>
...
</parameters>
</invoke>
</function_calls>

Here are the tools available:
<tools>

            <tool_description>
            <tool_name>get_weather</tool_name>
            <description>
            Returns weather data for a given latitude and longitude. </description>
            <parameters>
            <parameter>
            <name>latitude</name>
            <type>string</type>
            <description>The latitude coordinate as a string</description>
            </parameter> <parameter>
            <name>longitude</name>
            <type>string</type>
            <description>The longitude coordinate as a string</description>
         